# <img style="float: left; padding-right: 0.7cm; width: 3.0cm" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/179px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"> IMT 2210 - 1 Algebra lineal para Ciencias de Datos.
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2022-2**<br>
**Profesor** Cristobal rojas <br>

----

In [1]:
from vec import Vec, getitem, setitem, equal, add, dot, scalar_mul, neg
from listvec import ListVec
from math import sqrt

## Sistema Recomendador de Peliculas

### a) 

In [19]:
# Abrimos el archivo 'ratings.csv'
archivo = [i.strip().split(',') for i in list(open('ratings.csv'))]
# Primero seteamos algunos variables y objetos

archivo.pop(0)                    # Ignoramos los headers del archivo
n = archivo[0][0]                 # Primer id de usuario
function = dict()                 # Field del vector
movies_id = set()                 # Dominio del vector
users = dict()                    # Diccionario a retornar, con todos los vectores
for linea in archivo:
    movies_id.add(linea[1])

for linea in archivo:
    user_id = linea[0]
    if user_id == n:
        id_movie = linea[1]
        function[id_movie] = int(linea[2])
        user_id = linea[0]
    elif user_id != n:
        vec = Vec(movies_id, function)
        users[n] = vec
        n = user_id
        function = dict()

# Tenemos el diccionario de cada vector de gustos por usuario 

### b)

In [20]:
# Similar a la parte A

archivo = [i.strip().split(',') for i in list(open('ratings.csv'))]

archivo.pop(0)
n = archivo[0][0]
function = dict()
movies = dict()
users_id = set()

# Obtenemos el dominio de todos los vectores que sera el set de los ids de users
for linea in archivo:
    users_id.add(linea[0])

for linea in archivo:
    user_id = linea[0]
    if user_id == n:
        id_movie = linea[1]
        rating = linea[2]
        function[user_id] = int(rating)
    elif user_id != n:
        vec = Vec(users_id, function)
        movies[id_movie] = vec
        n = user_id
        function = dict()

### c) 

In [21]:
# En esta funcion ingresan dos vectores y se calcula la similaridad del coseno

def sim(u, v):
    sim = (dot(u, v)) / (sqrt(dot(u, u)) * sqrt(dot(v, v)))
    return sim



# Itero cada user de 'users' y calculo la similaridad para posteriormente almacenar en un diccionario 
# su id_user y la similitud respectiva con el user pedido, por último ordemnamos de manera decreciente
# para quedarnos y retornar con los primeros k

def vecinos(users, user_id, k):
    dict_simil = dict()
    for i in range(len(users)):
        if not users[str(i+1)] == users[user_id]:
            similitud = sim(users[str(i+1)], users[user_id])
            dict_simil[str(i+1)] = similitud
    orden = sorted(dict_simil.items(), key = lambda x:x[1], reverse=True)
    top_k = orden[:k]
    return top_k


#### Ejemplo parte c

In [23]:
# Prueba de la parte C
# tenemos 'users' (diccionario con los vectores gusto de cada usuario), parte A

# Ejemplo si queremos los 4 usuarios con gustos mas similares al usuario con id 25

vecinos_25 = vecinos(users, '25', 4)
print(vecinos_25)

[('913', 0.4118068187373492), ('748', 0.39172600297885474), ('650', 0.3799204168874656), ('882', 0.36837820085848094)]


(d) Responda: ¿Cómo se puede ocupar la informaci ́on obtenida en (c) para generar recomendaciones para
el usuario? Fundamente su respuesta. (4 pts.)

Se puede modelar un sistema de recomendaciones mas eficiente respecto a los gustos personales de la personas, teniendo a los usuarios con gustos similares, la funcion permite realizar diferenciación entre las películas y dar las mejores opciones de acuerdo a los gustos de cada usuario, a partir de eso crear una situacion en que se le presente solamente lo mejor primero o con prioridad de lo "ordenado".


(e) Reflexione: ¿cual es la ventaja de enfrentar este problema utilizando la clase Vec? ¿Que complicaciones
podrıa tener enfrentar este mismo problema utilizando la clase ListVec?

El mayor problema que podemos enfrentar a la hora de elegir una u otra clase es la eficiencia en la que se dara la recomendacion final y que luego de recolectara para un posible sistema de recomendados, ya que de alguna forma el vector que se dara podria tener una cantidad no querida de valores ceros finales (sparsity) que podria afectar la total recomendacion luego y no podria dar "algo" que uno quiere.

### Una aguja en un pajar

a) Definimos el método similarity en el archivo "listvec.py". 

b) Definimos la función "needle_search" siguiendo el algoritmo de comparación propuesto en la sección 2.9.3 del libro Coding the Matrix Ed.1

In [33]:
def needle_search(v: ListVec, u: ListVec):
    #Establecemos 2 contadores y 3 listas para generar los segmentos a analizar y sus valores de similaridad  respectivos.
    n = 0
    m = 0
    lista = []
    lista_seg = []
    lista_relacion = []
    
    # Si v > v
    if len(v) > len(u):
        # Identificamos cuales serán nuestros segmentos con el algoritmo propuesto en el libro.
        for i in range(len(v)-len(u)+1):
            for j in range(len(u)):
                lista.append(v[j+n])
            # Añadimos los segmentos generados y sus respectivos valores de similaridad en listas distintas.
            lista_seg.append(lista)
            lista_relacion.append(u.similarity(lista))
            lista = []
            n+=1
    # Si u > v
    else:
        # Identificamos cuales serán nuestros segmentos con el algoritmo propuesto en el libro.
        for i_1 in range(len(u)-len(v)+1):
            for j_1 in range(len(v)):
                lista.append(u[j_1+n])
            # Añadimos los segmentos generados y sus respectivos valores de similaridad en listas distintas.
            lista_seg.append(lista)
            lista_relacion.append(v.similarity(lista))
            lista = []
            n+=1
            
    # Popeamos elementos de la lista de relación y verificamos si son menores o iguales que el mínimo valor
    # de la misma para asegurarnos de tomar los elementos con mayor similaridad.       
    while len(lista_seg) != 3:
        x = lista_relacion.pop(m)
        if x <= min(lista_relacion):
            lista_seg.pop(m)
        m+=1
    
    # Transformamos los elementos de la lista a la clase ListVec y la retornamos
    lista_seg[0] = ListVec(lista_seg[0])
    lista_seg[1] = ListVec(lista_seg[1])
    lista_seg[2] = ListVec(lista_seg[2])
    return lista_seg

# Probamos la función para los vectores señalados en el ejemplo del libro.
pajar = ListVec([1, -1, 1, 1, 1, -1, 1, 1, 1])
aguja = ListVec([1, -1, 1, 1, -1, 1])

needle_search(aguja, pajar)

[[1, -1, 1, 1, 1, -1], [1, 1, 1, -1, 1, 1], [1, 1, -1, 1, 1, 1]]

c) La función descrita en el item (b) tiene múltiples utilidades de búsqueda, ya que nos facilita el análisis que se puede realizar en una serie de datos donde nos interese hallar segmentos con valores similares a los datos que nos interesa comprobar, esto nos sirve para describir muchas situaciones de la vida cotidiana, desde temas tan variados como analizar partes de de una canción hasta encontrar una aguja en un pajar, otros usos que podría tener serían la clasificación de datos categóricos para adecuarlos a los gustos usuarios que deseen buscar algo en concreto, ya sean series, películas, libros, etc.